In [1]:
%load_ext autoreload
%autoreload 2

In [18]:
sys.path.append('..')
from src.ham_to_sparse_matrix import *
from src.constants import *
from src.matrix_to_ps import matrix_to_pauli_strings
from src.binary_encodings import *

import sympy as sp
import sys
import scipy.sparse.linalg

In [19]:
bosonNI=sp.expand( pn**2/(2*aLat) + (aLat/2)*((qnP1-qnM1)/(2*aLat))**2 )
bosonI=sp.expand( (aLat/2)*V(qn)**2 + aLat*V(qn)*(qnP1-qnM1)/(4*aLat) + aLat*(qnP1-qnM1)*V(qn)/(4*aLat) )
fermionNI=sp.expand( -(xdnP1*xn+xdn*xnP1)/(2*aLat) )
fermionI=sp.expand( sp.diff(V(qn),qn)*(xdn*xn-(1/2)) )

In [20]:
N=3
aVal=1

# depends on finite-difference method
qs=[SiteSymbol('q',str(i)) for i in range(-1,N+1,1)]
# maybe make qs a normal site list
# and make an extra boundaryQs list for q[-1], q[N]
# counting would be normal computer science way for rest of code.

ps=[SiteSymbol('p',str(i)) for i in range(-1,N+1,1)] # don't really need extras
aops=[SiteSymbol('a',str(i)) for i in range(-1,N+1,1)]
adags=[SiteSymbol('a^{\dagger}',str(i)) for i in range(-1,N+1,1)]
xs=[SiteSymbol('\chi',str(i)) for i in range(-1,N+1,1)]
xdags=[SiteSymbol('\chi^{\dagger}',str(i)) for i in range(-1,N+1,1)]

# note this is exactly hardcoded for this finite difference method.
bcType = 'dirichlet'
boundaryConditions = {}
if bcType == 'periodic':
    boundaryConditions = {qs[0]: qs[N], qs[N+1]: qs[1],
                          xs[0]: -xs[N], xs[N+1]: -xs[1], 
                          xdags[0]: -xdags[N],  xdags[N+1]: -xdags[1]
                         }
    
elif bcType == 'dirichlet':
    boundaryConditions = { qs[0]: 0, qs[N+1]: 0,
                          xs[0]: 0, xs[N+1]: 0,
                          xdags[0]: 0, xdags[N+1]:0
                         }

In [21]:
ham=0

mass=1

for i in range(1,N+1):
    ham+=(bosonNI+bosonI+fermionNI).subs({
        pn: ps[i],
        qn: qs[i], qnP1: qs[i+1], qnM1: qs[i-1],
        xn: xs[i], xnP1: xs[i+1],
        xdn: xdags[i], xdnP1: xdags[i+1]
    }).subs(boundaryConditions)

def potential(n):
    # m*q with m=1
    return -mass*qs[n]

potentialSubs={}
for n in range(1,N+1):
    potentialSubs[V(qs[n])]=potential(n)


ham=sp.simplify(ham.subs(potentialSubs).subs(aLat,aVal))

for i in range(1,N+1):
    if i%2==0:
        ham+=sp.simplify(fermionI.subs({qn: qs[i], xn: xs[i], xdn: xdags[i]}).subs(potentialSubs))
    else:
        ham-=sp.simplify(fermionI.subs({qn: qs[i], xn: xs[i], xdn: xdags[i]}).subs(potentialSubs))
ham=sp.simplify(ham.subs(aLat,aVal))
sp.expand(ham)

-0.5 + \chi^{\dagger}_{0}*\chi_{0} - \chi^{\dagger}_{0}*\chi_{1}/2 - \chi^{\dagger}_{1}*\chi_{0}/2 - \chi^{\dagger}_{1}*\chi_{1} - \chi^{\dagger}_{1}*\chi_{2}/2 - \chi^{\dagger}_{2}*\chi_{1}/2 + \chi^{\dagger}_{2}*\chi_{2} + p_{0}**2/2 + p_{1}**2/2 + p_{2}**2/2 - q_{0}*q_{2}/8 + 5*q_{0}**2/8 + 3*q_{1}**2/4 - q_{2}*q_{0}/8 + 5*q_{2}**2/8

In [22]:
m=1

HOdofSubs = {}
#offset because of BC
for i in range(1,N+1):
    HOdofSubs[qs[i]] = 0.5*sp.sqrt(2/m)*(aops[i] + adags[i])
    HOdofSubs[ps[i]] = complex(0,1)*sp.sqrt(2*m)*(adags[i] - aops[i])/2 

hoHam=sp.expand(ham.subs(HOdofSubs))
hoHam=sp.nsimplify(hoHam,tolerance=1e-8)
hoHam

-1/2 + \chi^{\dagger}_{0}*\chi_{0} - \chi^{\dagger}_{0}*\chi_{1}/2 - \chi^{\dagger}_{1}*\chi_{0}/2 - \chi^{\dagger}_{1}*\chi_{1} - \chi^{\dagger}_{1}*\chi_{2}/2 - \chi^{\dagger}_{2}*\chi_{1}/2 + \chi^{\dagger}_{2}*\chi_{2} - a^{\dagger}_{0}*a^{\dagger}_{2}/16 + 9*a^{\dagger}_{0}*a_{0}/16 - a^{\dagger}_{0}*a_{2}/16 + a^{\dagger}_{0}**2/16 + 5*a^{\dagger}_{1}*a_{1}/8 + a^{\dagger}_{1}**2/8 - a^{\dagger}_{2}*a^{\dagger}_{0}/16 - a^{\dagger}_{2}*a_{0}/16 + 9*a^{\dagger}_{2}*a_{2}/16 + a^{\dagger}_{2}**2/16 + 9*a_{0}*a^{\dagger}_{0}/16 - a_{0}*a^{\dagger}_{2}/16 - a_{0}*a_{2}/16 + a_{0}**2/16 + 5*a_{1}*a^{\dagger}_{1}/8 + a_{1}**2/8 - a_{2}*a^{\dagger}_{0}/16 + 9*a_{2}*a^{\dagger}_{2}/16 - a_{2}*a_{0}/16 + a_{2}**2/16

In [74]:
cutoff=4
hamMat=convert_to_matrix(hoHam,cutoff,N,aops,adags,xs,xdags)

In [75]:
hamMat

<512x512 sparse matrix of type '<class 'numpy.complex64'>'
	with 3712 stored elements in Compressed Sparse Row format>

In [76]:
ens=scipy.sparse.linalg.eigs(hamMat,k=10,sigma=0.0)[0]
print(ens)

[1.17475014e-04+1.2479106e-13j 9.07817423e-01-3.5399005e-08j
 9.56369638e-01-6.2195658e-09j 1.00011611e+00+3.2164027e-10j
 1.03278172e+00+1.5129196e-09j 1.21190751e+00+1.0476639e-08j
 1.31557298e+00-1.0535340e-08j 1.56874096e+00-4.9989950e-08j
 1.22486067e+00+5.7678302e-09j 1.22486341e+00+1.4328487e-08j]


In [77]:
from src.qiskit_utilities import pauli_string_to_op
from qiskit import Aer
from qiskit.utils import QuantumInstance

from qiskit.algorithms import VQE
from qiskit.algorithms.optimizers import COBYLA
from qiskit.circuit.library import RealAmplitudes


backend = Aer.get_backend('statevector_simulator')
qinstance = QuantumInstance(backend, seed_simulator=2, seed_transpiler=2)


In [78]:
ps=matrix_to_pauli_strings(hamMat,standard_encode)
op = pauli_string_to_op(ps)

(0.23035518623524737+0j)


In [80]:
ansatz = RealAmplitudes((N**cutoff)*(N**2), reps=3)
vqe = VQE(ansatz=ansatz, optimizer=COBYLA(), quantum_instance=qinstance)
result = vqe.compute_minimum_eigenvalue(op)
print(result.eigenvalue)

(0.23087208062867912+0j)
